# Import packages and define sampling function

In [26]:
import pywikibot

import pandas as pd
import numpy as np

import random
random.seed(16021997)
from tqdm import tqdm
import concurrent.futures
from itertools import combinations

import requests
import bs4

# Scrape table with list of Wikipedias by number of articles

In [27]:
res = requests.get("https://meta.wikimedia.org/wiki/List_of_Wikipedias")
soup = bs4.BeautifulSoup(res.text, 'html.parser')

In [28]:
table = soup.find('table', class_='wikitable')
header_row = table.find('tr')
headers = [header.text.strip() for header in header_row.find_all('th')]
data_wikis = []

In [29]:
for row in table.find_all('tr'):
    columns = row.find_all('td')
    if columns:
        column_data = [column.text.strip() for column in columns]
        data_wikis.append(column_data)

In [30]:
df_wikis = pd.DataFrame(data_wikis, columns=headers)
df_wikis.drop('№',axis=1,inplace=True)
for col in ['Articles','All pages','Edits','Admins','Users','Active users','Files','Depth']:
    df_wikis[col] = df_wikis[col].str.replace(',', '')
    df_wikis[col] = df_wikis[col].astype(int)

In [31]:
df_wikis = df_wikis[(df_wikis.Wiki != 'ceb') & (df_wikis.Wiki != 'sv') & (df_wikis.Wiki != 'war')].reset_index(drop=True) # Dropping Cebuano, Swedish, and Waray

In [32]:
nr_articles_by_wiki = list(zip(df_wikis.Wiki,df_wikis.Articles))[:20]

In [33]:
wiki_language_mapping = dict(zip(df_wikis.Wiki,df_wikis.Language))

In [34]:
df_wikis.head(20)

,Language,Language (local),Wiki,Articles,All pages,Edits,Admins,Users,Active users,Files,Depth
0,English,English,en,6671092,58372321,1155971227,898,45721567,118218,904949,1189
1,German,Deutsch,de,2809964,7753780,233161769,180,4171486,17323,127771,93
2,French,français,fr,2529854,12605746,204799583,152,4682844,17753,71197,257
3,Dutch,Nederlands,nl,2124832,4540641,64351059,34,1282317,3853,20,18
4,Russian,русский,ru,1922650,7716154,130818350,72,3405051,10550,243991,153
5,Spanish,español,es,1870941,7930279,151456769,58,6876883,15831,0,200
6,Italian,italiano,it,1815042,7741567,133725350,120,2394740,8194,131150,184
7,Egyptian Arabic,مصرى,arz,1617928,2074631,8155224,7,212866,217,1482,0
8,Polish,polski,pl,1571240,3620567,70458286,100,1243567,4307,260,33
9,Japanese,日本語,ja,1377456,4054827,95460294,42,2082977,14126,4622,88


# Extract language connections from a sample of pages

Each Wiki will be sampled in proportion to its # of articles.

In [35]:
rescaling_factor = 0.03 # We'll sample only this proportion of pages from each wiki

In [36]:
'''dict_data = {}
for wiki_version, nr_articles in tqdm(nr_articles_by_wiki, desc='Outer loop'):
    
    site = pywikibot.Site(wiki_version, "wikipedia") 
    sampled_articles = list(site.randompages(total=max(int(nr_articles * rescaling_factor),1),namespaces=[0])) # we want to grab at least one article per Wiki 
    
    for article in tqdm(sampled_articles, desc='Inner loop'):
        
        page = pywikibot.Page(site, article.title())
        
        langlinks = [langlink.site.lang for langlink in page.langlinks()]
        dict_data[f'{wiki_version}-{article.title()}'] = langlinks + wiki_version'''

'dict_data = {}\nfor wiki_version, nr_articles in tqdm(nr_articles_by_wiki, desc=\'Outer loop\'):\n    \n    site = pywikibot.Site(wiki_version, "wikipedia") \n    sampled_articles = list(site.randompages(total=max(int(nr_articles * rescaling_factor),1),namespaces=[0])) # we want to grab at least one article per Wiki \n    \n    for article in tqdm(sampled_articles, desc=\'Inner loop\'):\n        \n        page = pywikibot.Page(site, article.title())\n        \n        langlinks = [langlink.site.lang for langlink in page.langlinks()]\n        dict_data[f\'{wiki_version}-{article.title()}\'] = langlinks + wiki_version'

In [37]:
def process_article(site, article):
    page = pywikibot.Page(site, article.title())
    langlinks = [langlink.site.lang for langlink in page.langlinks()]
    return f'{site.code}-{article.title()}', langlinks + [site.code]

In [38]:
dict_data = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    for wiki_version, nr_articles in tqdm(nr_articles_by_wiki, desc='Outer loop'):
        site = pywikibot.Site(wiki_version, "wikipedia")
        sampled_articles = list(site.randompages(total=max(int(nr_articles * rescaling_factor), 1), namespaces=[0])) # We want to select at least one article per Wiki

        future_to_article = {executor.submit(process_article, site, article): article for article in sampled_articles}

        for future in tqdm(concurrent.futures.as_completed(future_to_article), desc='Inner loop', total=len(future_to_article)):
            article = future_to_article[future]
            result = future.result()
            dict_data[result[0]] = result[1]

Inner loop:   1%|▏                        | 1017/200132 [00:17<51:24, 64.56it/s]


Inner loop:   1%|▎                        | 2028/200132 [00:33<44:19, 74.50it/s]


Inner loop:   2%|▍                        | 3076/200132 [00:49<46:29, 70.64it/s]


Inner loop:   2%|▌                        | 4129/200132 [01:07<49:14, 66.35it/s]


Inner loop:   3%|▋                        | 5132/200132 [01:23<44:40, 72.76it/s]


...

Inner loop:  58%|█████████████▍         | 116484/200132 [34:25<22:10, 62.85it/s]


Inner loop:  59%|█████████████▌         | 117497/200132 [34:43<20:08, 68.36it/s]


Inner loop:  59%|█████████████▌         | 118435/200132 [35:00<20:15, 67.24it/s]


Inner loop:  60%|█████████████▋         | 119170/200132 [35:12<36:03, 37.42it/s]ERROR: Traceback (most recent call last):
  File "/Users/federico.bindi/anaconda3/envs/baseplus/lib/python3.10/site-packages/pywikibot/data/api/_requests.py", line 681, in _http_request
    response = http.request(self.site, uri=uri,
  File "/Users/federico.bindi/anaconda3/envs/baseplus/lib/python3.10/site-packages/pywikibot/comms/http.py", line 282, in request
    r = fetch(baseuri, headers=headers, **kwargs)
  File "/Users/federico.bindi/anaconda3/envs/baseplus/lib/python3.10/site-packages/pywikibot/comms/http.py", line 448, in fetch
    callback(response)
  File "/Users/federico.bindi/anaconda3/envs/baseplus/lib/python3.10/site-packages/pywikibot/comms/http.py", line 327, in error_handling_callback
    raise ConnectionError(response)
ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?titles=Mount+Marcy+%28disambiguation%29&prop=langlinks&act

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?titles=Mount+Marcy+%28disambiguation%29&prop=langlinks&action=query&indexpageids=&continue=&lllimit=500&meta=userinfo&uiprop=blockinfo%7Chasmsg&maxlag=5&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2ade17e50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

# Prepare network data (size of nodes and links) and save it

Create df of node sizes:

In [ ]:
df_wikis.to_csv('Node sizes.csv')

Create df of language pairs:

In [ ]:
value_combinations = list(combinations(my_dict.values(), 2))
rows = [(value1, value2) if value1 <= value2 else (value2, value1) for value1, value2 in value_combinations]
df_connections = pd.DataFrame(rows, columns=['lang_1','lang_2'])
df_connections.drop_duplicates(inplace=True)
df_connections.reset_index(drop=True, inplace=True)

In [ ]:
#df_connections.to_csv('Edges.csv')